In [33]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets, transforms
import os
import pickle

from torch.utils.data import DataLoader

In [34]:
path = os.getcwd()

In [35]:
ls

a.ipynb          test/            val/
col775-a1.ipynb  train/


In [41]:
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=25):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2])

model = ResNet18()


In [42]:
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

# Assuming device is available (either "cuda" or "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and scheduler
optimizer = SGD(model.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

# Loss function
criterion = nn.CrossEntropyLoss()

def calculate_metrics(preds, labels):
    preds = np.argmax(preds, axis=1)
    acc = accuracy_score(labels, preds)
    micro_f1 = f1_score(labels, preds, average='micro')
    macro_f1 = f1_score(labels, preds, average='macro')
    return acc, micro_f1, macro_f1

# Training and validation loops
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    train_losses = []
    val_losses = []
    # For storing predictions and labels for calculating metrics
    train_preds, train_labels = [], []
    val_preds, val_labels = [], []

    # Training loop
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())
        train_preds.extend(outputs.cpu().detach().numpy())
        train_labels.extend(labels.cpu().detach().numpy())

    # Validation loop
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_losses.append(loss.item())
            val_preds.extend(outputs.cpu().detach().numpy())
            val_labels.extend(labels.cpu().detach().numpy())

    # Calculate metrics
    train_acc, train_micro_f1, train_macro_f1 = calculate_metrics(np.array(train_preds), np.array(train_labels))
    val_acc, val_micro_f1, val_macro_f1 = calculate_metrics(np.array(val_preds), np.array(val_labels))

    # Adjust the learning rate based on the scheduler
    scheduler.step()

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {np.mean(train_losses):.4f}, Val Loss: {np.mean(val_losses):.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Train Micro F1: {train_micro_f1:.4f}, Val Micro F1: {val_micro_f1:.4f}, Train Macro F1: {train_macro_f1:.4f}, Val Macro F1: {val_macro_f1:.4f}')

# You should also implement code to plot the error, accuracy, and F1 scores as required.


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x16384 and 64x25)

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use t

RuntimeError: The size of tensor a (128) must match the size of tensor b (256) at non-singleton dimension 3